In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import face_alignment
import os

In [2]:
data_dir = '/DATA5_DB8/data/zdcheng/hyperthyreosis_eye/TAO_face/夏妍妍/外观'

In [3]:
def pad(img,padding_ratio):
    h,w,_ = img.shape
    h,w = (int(h*padding_ratio),int(w*padding_ratio))
    return cv2.copyMakeBorder(img,h,h,w,w,cv2.BORDER_CONSTANT)

In [4]:
def process_dir(data_dir,padding_ratio,input_ratio):
    
    out_dir = data_dir + '_out/standard'
    pad_out_dir = data_dir + '_out/pad'
    imgs = os.listdir(data_dir)
    imgs = [i for i in imgs if i.endswith('jpg') or i.endswith('JPG')]
#     imgs = [os.path.join(data_path,i) for i in imgs if i.endswith('jpg') or i.endswith('JPG')]
    fa = face_alignment.FaceAlignment(face_alignment.LandmarksType._2D, flip_input=False)
    for i in imgs:
        img_path = os.path.join(data_dir,i)
        out_path = os.path.join(out_dir,i)
        pad_out_path = os.path.join(pad_out_dir,i)
        if os.path.exists(out_path) and os.path.exists(pad_out_path):
            continue
        
        img = cv2.imread(img_path)
        img_padded = pad(img,padding_ratio)
        img_padded_RGB = cv2.cvtColor(img_padded, cv2.COLOR_BGR2RGB)
        
        padded_h,padded_w,_ = img_padded_RGB.shape
        input_h,input_w = int(padded_h*input_ratio),int(padded_w*input_ratio)
        input_RGB = cv2.resize(img_padded_RGB,(input_w,input_h))
        
        pred = fa.get_landmarks(input_RGB)[0]
        
        added_h,added_w = int(img.shape[0]*padding_ratio),int(img.shape[1]*padding_ratio)
        pred_ori = pred/input_ratio
        pred_ori[:,0] = pred_ori[:,0] - added_w
        pred_ori[:,1] = pred_ori[:,1] - added_h
        
        pad_out = cv2.cvtColor(input_RGB,cv2.COLOR_RGB2BGR)
        ori_out = img.copy()
        
        for p in pred:
            cv2.circle(pad_out,(p[0],p[1]),6,(255,255,255),-1)
        for p in pred_ori:
            cv2.circle(ori_out,(p[0],p[1]),20,(255,255,255),-1)
        if not os.path.exists(pad_out_path):
            cv2.imwrite(pad_out_path,pad_out)
        if not os.path.exists(out_path):
            cv2.imwrite(out_path,ori_out)

In [5]:
process_dir(data_dir,padding_ratio=1, input_ratio=0.15)

In [6]:
cv2.imwrite('/DB/rhome/zdcheng/tmp.JPG',np.zeros((3,3,3),dtype = 'uint8'))

True